# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=33

In [3]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==33]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm33', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm33/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale
2017-08-27 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26601/3795637845.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26601/3795637845.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26601/3795637845.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"


R[write to console]: Error in (function (f, ts, currdate, out_dir, bandwidth, mon_low, mon_up)  : 
  Too few data points!

R[write to console]: In addition: 
R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



Sim failed, proceeding with the next one... -  ['oldschool']
exception:  Error in (function (f, ts, currdate, out_dir, bandwidth, mon_low, mon_up)  : 
  Too few data points!

2017-11-26 00:00:00
[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"


...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]      [,4]
sigmas -0.637335684 1.0939418 -1.205455929 0.7138325
       -0.626308785 1.0871270 -1.205098140 0.7584751
       -0.615402154 1.0802932 -1.204323355 0.8021961
       -0.604613195 1.0734435 -1.203140093 0.8450005
       -0.593939396 1.0665814 -1.201556862 0.8868938
       -0.583378325 1.0597100 -1.199582151 0.9278813
       -0.572927626 1.0528324 -1.197224426 0.9679689
       -0.562585015 1.0459518 -1.194492134 1.0071625
       -0.552348280 1.0390711 -1.191393688 1.0454683
       -0.542215275 1.0321934 -1.187937476 1.0828927
       -0.532183918 1.0253214 -1.184131849 1.1194423
       -0.522252191 1.0184581 -1.179985118 1.1551239
       -0.512418133 1.0116061 -1.175505558 1.1899445
       -0.502679844 1.0047682 -1.170701397 1.2239113
       -0.493035474 0.9979469 -1.165580814 1.2570318
       -0.483483231 0.9911449 -1.160151941 1.2893135
       -0.474021371 0.9843645 -1.154422852 1.3207642
       -0.46

        0.350480494 0.6447961  0.336802856 1.6791307
        0.354601013 0.6468010  0.346512473 1.6725913
        0.358704622 0.6488622  0.356181254 1.6660388
        0.362791461 0.6509795  0.365809174 1.6594735
        0.366861665 0.6531527  0.375396204 1.6528957
        0.370915370 0.6553815  0.384942318 1.6463055
        0.374952709 0.6576657  0.394447485 1.6397032
        0.378973813 0.6600050  0.403911674 1.6330889
        0.382978813 0.6623992  0.413334855 1.6264630
        0.386967836 0.6648481  0.422716994 1.6198255
        0.390941011 0.6673514  0.432058059 1.6131767
        0.394898462 0.6699089  0.441358016 1.6065168
        0.398840313 0.6725203  0.450616831 1.5998458
        0.402766687 0.6751853  0.459834468 1.5931640
        0.406677704 0.6779039  0.469010893 1.5864716
        0.410573486 0.6806756  0.478146070 1.5797686
        0.414454149 0.6835003  0.487239966 1.5730553
        0.418319810 0.6863777  0.496292546 1.5663317
        0.422170586 0.6893075  0.505303775 1.5

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26601/3795637845.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26601/3795637845.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_26601/3795637845.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]          [,4]
sigmas -0.638705483 1.1055428 -1.908828778  3.3105567326
       -0.626929637 1.0941347 -1.893393564  3.3351512541
       -0.615290850 1.0827943 -1.877638938  3.3589067131
       -0.603785967 1.0715244 -1.861572472  3.3818240451
       -0.592411942 1.0603281 -1.845201724  3.4039041454
       -0.581165833 1.0492081 -1.828534241  3.4251478759
       -0.570044793 1.0381670 -1.811577562  3.4455560718
       -0.559046071 1.0272077 -1.794339217  3.4651295487
       -0.548167007 1.0163328 -1.776826729  3.4838691098
       -0.537405024 1.0055447 -1.759047617  3.5017755529
       -0.526757629 0.9948461 -1.741009398  3.5188496787
       -0.516222407 0.9842394 -1.722719587  3.5350922973
       -0.505797021 0.9737269 -1.704185696  3.5505042370
       -0.495479203 0.9633110 -1.685415240  3.5650863515
       -0.485266756 0.9529940 -1.666415735  3.5788395286
       -0.475157549 0.9427781 -1.647194699  3.59176469

        0.347877590 0.6063231  0.387496788  0.4886701232
        0.352284425 0.6087561  0.390428918  0.4504069422
        0.356671926 0.6112063  0.393128968  0.4122351368
        0.361040260 0.6136722  0.395598153  0.3741580839
        0.365389595 0.6161525  0.397837708  0.3361790854
        0.369720095 0.6186459  0.399848884  0.2983013714
        0.374031923 0.6211508  0.401632954  0.2605281042
        0.378325238 0.6236661  0.403191205  0.2228623820
        0.382600200 0.6261904  0.404524946  0.1853072425
        0.386856964 0.6287223  0.405635500  0.1478656661
        0.391095685 0.6312606  0.406524209  0.1105405796
        0.395316515 0.6338039  0.407192435  0.0733348594
        0.399519605 0.6363510  0.407641556  0.0362513347
        0.403705102 0.6389006  0.407872966 -0.0007072094
        0.407873155 0.6414513  0.407888081 -0.0375380289
        0.412023906 0.6440021  0.407688333 -0.0742384183
        0.416157500 0.6465515  0.407275173 -0.1108057072
        0.420274078 0.6490984  

In [4]:
print(error_i)

[0, 2, 3, 4, 6, 8, 12, 14, 15, 18]
